### This IPython notebook defines several variations of convolutional neural networks for channel estimation. The training inputs are the preamble + preamble passed through channel; the predicted output is the channel taps that correspond to the input. We explore several ideas here:
#### (A) multi-scale convolution (learned) filters applied separately to the [preamble input] and to the [preamble thorugh channel] input
#### (B) multi-scale convolution (learned) filters applied to both (e.g., 2D convolution filters)

We make several assumptions about the channel model here as well:
* Channel length is <= 20
* Channel energy (am I saying this correctly?) is 1 (also, does normalizing channel taps by l2 norm ensure this?)
* Channel is sparse (most entries near 0, except for a few spikes)
  * Potential simplifying assumption (maybe include initially?) first entry of channel is 'large'
  
  
Questions: 
1. for my preamble, I am using +/- 1; Nikhil used 1/0 .. which is correct? (It should not matter really for training/testing since it is a simple affine transform between the two, but I want to do the "correct" thing)
2. do my assumptions make sense? for a real model I mean
3. am I adding noise correctly for the SNR I am setting
4. More of a "TODO" but...I am only training and testing on preamble inputs, not additional data -- the reasoning is that for additional data, we really want something that handles sequences (e.g., and RNN) in my opinion and this is more of an exploratoration of convolutional layers here

## ALSO NOTE: I am making a lot of things very modular on purpose..I want to discuss with everyone the problem statement again (I still feel like a lot of things are unclear/ambiguous) and then we can move a lot of this modular code to a rigid "util.py" file that everyone should import from so that we can more easily guarantee correctness and consistency and speed up development time.

In [1]:
# standard imports
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [100]:
# utility functions...we really should standardize this in a Python file [TODO!!!!]
"""Generates random sequence [1 1 1 -1 1 -1 -1 ...] of length LENGTH."""
def gen_preamble(length=100):
    return np.random.randint(2, size=(1,length))*2 - 1

"""Generates N channels of length LENGTH, each with NUM_TAPS taps. This
   means that NUM_TAPS of the entries will be non-zero, and the rest will
   be 'close' to 0 (e.g., noise). 
   Example below.
   
   >>> np.around(gen_channel(),2)
   >>> array([[-0.08,  0.  , -0.06,  0.02,  0.  ,  0.02, -0.85,  0.05, -0.03,
        -0.07,  0.5 , -0.02, -0.  , -0.05, -0.  ,  0.03, -0.07, -0.04,
        -0.01,  0.08]])"""
def gen_channel(N=1,num_taps=2,length=20):
    ret = np.zeros((N, length))
    tap_idxs = np.random.randint(length, size=(N, num_taps))
    tap_vals = ((np.random.randint(10, size=(N, num_taps))+1)*\
                (np.random.randint(2, size=(N, num_taps))*2 - 1))\
                / 10.
    for i in range(N):
        np.put(ret[i], tap_idxs[i], tap_vals[i])
    ret += 5e-2*np.random.randn(N,length)
    return ret / np.linalg.norm(ret,axis=1,keepdims=True)

"""Simulates passing data through a noisy channel.
   If SNR == -1, then no noise. Otherwise, uses AWGN model.
   
   Returned value has shape (1, len(channel.T) + len(data.T) - 1).
   With default settings, this means it is (1, 119)."""
def apply_channel(channel, data, snr=-1):
    ret = sig.convolve(data, channel, mode='full')
    if snr > 0:
        ret += (1./np.sqrt(snr)) * np.random.randn(len(ret))
    return ret

In [94]:
# functions for networks..should also put this in util.py!
"""Run before building a new network. Rests randomization for repeatability."""
def reset():
    tf.reset_default_graph()
    np.random.seed(0)
    tf.set_random_seed(0)
    
"""Defines the loss function."""
def define_loss(placeholders, loss_type):
    output, correct_output = placeholders
    return tf.reduce_mean(tf.reduce_sum((output-correct_output)**2, axis=1))
    
"""Defines the optimizer."""
def define_optimizer(loss, trainable_weights, optimizer, lr):
    opt = tf.train.AdamOptimizer(lr)
    gradients = opt.compute_gradients(loss, trainable_weights)
    train_step = opt.apply_gradients(gradients)
    return train_step

"""Defines a trainable variable with truncated normal initialization."""
def define_variable(name, shape, stddev):
    var = tf.get_variable(name, shape, initializer=
                    tf.truncated_normal_initializer(stddev=stddev, dtype=tf.float32),
                    dtype=tf.float32)
    return var

In [158]:
# define the networks
"""Builds the network [model 1] -- a basic convolution network; use as a base
   for the next network models.
   Elements in PARAMS:
   
   * 'preamble_len' : length of preamble; [default = 100]
   * 'channel_len' : length of channel; [default = 20]
   * 'use_max_pool': True to use max pooling in first part of net; [default = False]
   * 'loss' : loss function to use
   * 'optimizer' : optimizer to use
   * 'lr' : base learning rate
   
   """
def build_network1(params=None):
    if params == None:
        params = {'preamble_len':100, 'channel_len':20,
                  'use_max_pool':False, 'loss':"", 'optimizer':"", 'lr':4e-5}
    preamble = tf.placeholder(tf.float32, [1, params['preamble_len'], 1], name="preamble_input")
    # use same length as preamble as per discussion on April 12
    received = tf.placeholder(tf.float32, [None, params['preamble_len'], 1], name="received_preamble")
    channel_true = tf.placeholder(tf.float32, [None, params['channel_len']])
    batch_size = tf.shape(received)[0]
    
    inputs=[preamble,received,channel_true]
    outputs=[]
    weights=[]
    
    nets=[preamble,received]
    
    # Process PREAMBLE and RECEIVED separately through convolutions
    num_filters = [1, 30, 30, 10]
    for i in [1,2]:
        net = nets[i-1]
        for j in range(1, len(num_filters)):
            num_filter = num_filters[j]
            prev = num_filters[j-1]
            with tf.variable_scope("conv%d_%d" % (j+1, i)) as scope:
                # use same weight initializer for all, and always use 3x_ convolutions
                kernel = define_variable('conv_weights', [3, prev, num_filter], 5e-2)
                biases = define_variable('conv_biases', [num_filter], 5e-3)
                weights.extend([kernel, biases])
                # apply network
                net = tf.nn.conv1d(net, kernel, stride=1, padding='SAME')
                net = tf.nn.bias_add(net, biases)
                net = tf.nn.relu(net)
                if params['use_max_pool']:
                    net = tf.nn.max_pool(net, [1, 3, 1], [1, 2, 1], padding='SAME')
        nets[i-1] = net
        
    # Concatenate
    nets[0] = tf.tile(nets[0], [batch_size, 1, 1])
    output = tf.concat(nets, axis=1)
    with tf.variable_scope("conv1_concat") as scope:
        kernel = define_variable('conv_weights', [3, num_filters[-1], 10], 5e-2)
        biases = define_variable('conv_biases', [10], 5e-3)
        weights.extend([kernel, biases])
        # apply network
        net = tf.nn.conv1d(net, kernel, stride=1, padding='SAME')
        net = tf.nn.bias_add(net, biases)
        net = tf.nn.relu(net)
    with tf.variable_scope("fc2_concat") as scope:
        dim = output.get_shape()[1].value*output.get_shape()[2].value
        batch_size = tf.shape(output)[0]
        
        kernel = define_variable('conv_weights', [dim, params['channel_len']], 5e-2)
        biases = define_variable('conv_biases', [params['channel_len']], 5e-3)
        weights.extend([kernel, biases])
        # apply network
        output = tf.reshape(output, [batch_size, -1])
        output = tf.nn.sigmoid(tf.matmul(output, kernel) + biases)
    
    outputs=[output]
    
    loss = define_loss([output, channel_true], params['loss'])
    train = define_optimizer(loss, weights, params['optimizer'], params['lr'])

    return inputs, outputs, weights, loss, train

In [159]:
# train network
reset()
inputs, outputs, weights, loss, train = build_network1()
num_iter=10000
batch_size=100
# use a single fixed preamble
preamble=gen_preamble()
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
for i in range(0,num_iter):
    # generate data
    channels = gen_channel(N=batch_size)
    received = apply_channel(channels, preamble, snr=-1)
    #channels = channels.reshape((batch_size,-1,1))
    received = received.reshape((batch_size,-1,1))[:,:100,:]
    
    # train
    sess.run(train, feed_dict={inputs[0]:preamble.reshape((1, 100, 1)),
                     inputs[1]:received,
                     inputs[2]:channels})
    if i % 100 == 0:
        l = sess.run(loss, feed_dict={inputs[0]:preamble.reshape((1, 100, 1)),
                     inputs[1]:received,
                     inputs[2]:channels})
        print(i,l)

0 6.0226917
10 6.0540795
20 6.153272
30 5.959567
40 6.021763
50 5.864655
60 5.6800714
70 5.734802
80 5.6470795
90 5.625172
100 5.304321
110 5.3177366
120 4.884723
130 4.5628405
140 4.402691
150 3.9965124
160 3.604062
170 3.291089
180 2.796199
190 2.4410746
200 2.147702
210 1.8360114
220 1.6814902
230 1.5715724
240 1.4261998
250 1.3141888
260 1.2320234
270 1.2061732
280 1.1661541
290 1.1247128
300 1.1252065
310 1.1267849
320 1.0958221
330 1.0722934
340 1.054806
350 1.0763851
360 1.0445817
370 1.0490582
380 1.047296
390 1.0449301
400 1.0428039
410 1.0282346
420 1.028836
430 1.0342497
440 1.0379522
450 1.0285667
460 1.0359975
470 1.0195144
480 1.0244099
490 1.0278993
500 1.0182813
510 1.0241394
520 1.0141172
530 1.0180893
540 1.0145458
550 1.014996
560 1.0219283
570 1.0192297
580 1.0140438
590 1.0111045
600 1.0115474
610 1.008149
620 1.011463
630 1.0092558
640 1.0146624
650 1.0041976
660 1.007657
670 1.0085491
680 1.0059404
690 1.0120081
700 1.0077308
710 1.0080199
720 1.0091538
730 1.002

5510 0.8717929
5520 0.8809567
5530 0.86067826
5540 0.8454764
5550 0.79631287
5560 0.80154556
5570 0.81537473
5580 0.80722255
5590 0.7527994
5600 0.77116257
5610 0.7439624
5620 0.78887963
5630 0.75524384
5640 0.7458747
5650 0.74857676
5660 0.720375
5670 0.7247296
5680 0.7409442
5690 0.72958267
5700 0.7505877
5710 0.723272
5720 0.6909953
5730 0.70705795
5740 0.712134
5750 0.6491242
5760 0.71918696
5770 0.6584153
5780 0.6844046
5790 0.6233023
5800 0.6403683
5810 0.6317619
5820 0.59011596
5830 0.63512975
5840 0.63101333
5850 0.6128415
5860 0.6224543
5870 0.60973895
5880 0.59630716
5890 0.5388184
5900 0.6227328
5910 0.6085026
5920 0.5371153
5930 0.6521257
5940 0.544988
5950 0.5401281
5960 0.58045447
5970 0.6076745
5980 0.5417662
5990 0.54883504
6000 0.60136104
6010 0.53466535
6020 0.53872156
6030 0.5774787
6040 0.5540005
6050 0.56299174
6060 0.5720539
6070 0.60972035
6080 0.60624593
6090 0.5923791
6100 0.59796286
6110 0.53248954
6120 0.59962523
6130 0.5622322
6140 0.5581155
6150 0.54527384


In [123]:
preamble=gen_preamble()
channels = gen_channel(N=30)
received = apply_channel(channels, preamble, snr=-1)
print(preamble.shape, channels.shape, received.shape)

(1, 100) (30, 20) (30, 119)


In [134]:
for i in range(3):
    print (i, inputs[i].shape)

0 (1, 100, 1)
1 (?, 100, 1)
2 (?, 20)


In [139]:
print (channels.shape)

(100, 20)


In [138]:
print(received.shape)

(100, 100, 1)
